# Buscador de imágenes por descripción
## Autores
1. Jhonny Cerezo
2. Jesús Pérez

## Introduction
Las redes neuronales son bien conocidas por problemas de clasificación, por ejemplo, se usan en la clasificación de dígitos escritos a mano, pero la pregunta es ¿será fructífero si los usamos para problemas de regresión?

En este trabajo usaremos las redes neuronales para representar descripciones textuales de imágenes de COCO en el espacio de los descriptores visuales de las imágenes de 2048 dimenciones obtenidos mediante ResNet152. Con esto pretendemos realizar búsquedas de las imágenes relacionadas con una descripción.

## Requirements
1. Python 3.6
2. Pytorch 1.1.0

In [ ]:
!virtualenv -p python3 venv3
!pip install -r requirements.txt
# !pip install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
# !pip install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl
# !pip install tensorboardX
# !pip install scikit-learn
# !pip install nltk
# !pip install gensim

## Import packages

In [1]:
import os
import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from tensorboardX import SummaryWriter
import numpy as np

## Download pretrained word embeddings
Para la representacion de las descipciones en Español de las imágenes se puede usar word-embeddings. Estos embeddings representan espacios donde ocurren propiedades interesantes entre las representaciones de cada una de las palabras. En nuestros experimentos word-embeddings preentrenados basados en: FastText y GloVe. 

In [3]:
if not os.path.exists('./word-embeddings'):
    os.mkdir('word-embeddings')
    # download word-embeddings based on FastText
    !cd word-embeddings && wget http://dcc.uchile.cl/~jperez/word-embeddings/fasttext-sbwc.vec.gz && gunzip fasttext-sbwc.vec.gz
    # download word-embeddings based on GloVe
    !cd word-embeddings && wget http://dcc.uchile.cl/~jperez/word-embeddings/glove-sbwc.i25.vec.gz && gunzip glove-sbwc.i25.vec.gz

## Dataset
1. Download the COCO-2014-Spanish from https://drive.google.com/drive/folders/1RzGYR2uqMRS4WqX_wqIiI2Y_NdNAey1m

In [2]:
# path to the dataset folder
base_dir = '/data/jeperez/COCO-2014-spanish/'

In [3]:
from data import get_loader, load_coco_files

phases = ['train', 'test_A', 'test_B', 'test_C']
loaders, coco_images_names, coco_visual_feats, coco_captions = {}, {}, {}, {}
for phase in phases:
    print('\n{}'.format(phase))
    folder_dir = os.path.join(base_dir, phase)
    file_names = os.path.join(folder_dir, '{}_images_names.txt'.format(phase))
    file_vectors = os.path.join(folder_dir, '{}_images_vectors.bin'.format(phase))
    file_captions = os.path.join(folder_dir, '{}_captions.txt'.format(phase))
    coco_images_names[phase], coco_visual_feats[phase], coco_captions[phase] = load_coco_files(file_names, file_vectors, file_captions, 2048)
            
train_names, train_texts = zip(*coco_captions['train'])
print('\nsample train data')
for s in ['{}: {}'.format(n, s) for n,s in zip(train_names[0:10], train_texts[0:10])]:
    print(s)


train
leyendo /data/jeperez/COCO-2014-spanish/train/train_images_names.txt
leyendo /data/jeperez/COCO-2014-spanish/train/train_images_vectors.bin
20000 vectores de largo 2048
leyendo /data/jeperez/COCO-2014-spanish/train/train_captions.txt

test_A
leyendo /data/jeperez/COCO-2014-spanish/test_A/test_A_images_names.txt
leyendo /data/jeperez/COCO-2014-spanish/test_A/test_A_images_vectors.bin
1000 vectores de largo 2048
leyendo /data/jeperez/COCO-2014-spanish/test_A/test_A_captions.txt

test_B
leyendo /data/jeperez/COCO-2014-spanish/test_B/test_B_images_names.txt
leyendo /data/jeperez/COCO-2014-spanish/test_B/test_B_images_vectors.bin
1000 vectores de largo 2048
leyendo /data/jeperez/COCO-2014-spanish/test_B/test_B_captions.txt

test_C
leyendo /data/jeperez/COCO-2014-spanish/test_C/test_C_images_names.txt
leyendo /data/jeperez/COCO-2014-spanish/test_C/test_C_images_vectors.bin
1000 vectores de largo 2048
leyendo /data/jeperez/COCO-2014-spanish/test_C/test_C_captions.txt

sample train data

## Define the divice to be used
Seleccionamos el GPU con mayor espacio libre. Si desea correr los experimentos en CPU, cambie el valor de la variable 'device'

In [4]:
from utils import get_freer_gpu

device = 'gpu'

if device == 'gpu' and torch.cuda.is_available():
    freer_gpu_id = get_freer_gpu()
    device = torch.device('cuda:{}'.format(freer_gpu_id))
    torch.cuda.empty_cache()
else:
    device = torch.device('cpu')

print(device)

cuda:0


## Initialize Text Descriptor

In [12]:
text_descriptor_name = 'GloVe'

if text_descriptor_name == 'tf-idf':
    from text_descriptors.bow import TextDescriptor
    text_descriptor = TextDescriptor(type='tf-idf', texts=train_texts, lowecase=False, ngram_range=(1,3), 
                                     max_df=.8, min_df=.01)
# if text_descriptor_name == 'bow':
#     from text_descriptors.bow import TextDescriptor
#     text_descriptor = TextDescriptor(type='bow', texts=train_texts, lowecase=False, ngram_range=(1,1), 
#                                      max_df=.8, min_df=.01)
# elif text_descriptor_name == 'lsa':
#     from text_descriptors.lsa import LSADescriptor
#     text_descriptor = LSADescriptor(type='tf-idf', texts=train_texts, lowecase=False, ngram_range=(1,3), 
#                                     max_df=.8, min_df=.01, n_components=100)
elif text_descriptor_name == 'FastText':
    from text_descriptors.embedding import WordEmbedding
    from gensim.models.keyedvectors import KeyedVectors
    wordvectors_file_vec = './word-embeddings/fasttext-sbwc.vec'
    cantidad = 100000
    wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)
    text_descriptor = WordEmbedding(wordvectors, embedding_dim=300)
elif text_descriptor_name == 'GloVe':
    from text_descriptors.embedding import WordEmbedding
    from gensim.models.keyedvectors import KeyedVectors
    wordvectors_file_vec = './word-embeddings/glove-sbwc.i25.vec'
    cantidad = 100000
    wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)
    text_descriptor = WordEmbedding(wordvectors, embedding_dim=300)
# elif text_descriptor_name == 'my-embedding':
#     from text_descriptors.embedding import WordEmbedding
#     from vocabulary import Vocabulary
#     vocab = Vocabulary(max_df=1, min_df=0)
#     vocab.add_sentences(train_texts)
#     vocab.add_words(['<unk>', '<pad>'])
#     text_descriptor = WordEmbedding(num_embeddings=len(vocab), embedding_dim=300).to(device)
else:
    raise 'unknown descriptor {}'.format(text_descriptor_name)

print('descriptor size: {}'.format(text_descriptor.out_size))
print(text_descriptor.transform(['hermosa Habana', 'hola Cuba']).shape)

descriptor size: 300
(2, 300)


## Initialize Data Loaders

In [13]:
batch_size = {'train': 200, 'test_A': 1000, 'test_B': 1000, 'test_C': 1000}
shuffle = {'train': True, 'test_A': False, 'test_B': False, 'test_C': False}
num_workers = {'train': 4, 'test_A': 1, 'test_B': 1, 'test_C': 1}
pin_memory = {'train': True, 'test_A': False, 'test_B': False, 'test_C': False}

loaders = {}
for phase in phases:
    phase_names, phase_captions = zip(*coco_captions[phase])
    phase_captions_feats = text_descriptor.transform(phase_captions)
    loaders[phase] = get_loader(coco_images_names[phase], coco_visual_feats[phase], coco_captions[phase], phase_captions_feats, batch_size[phase], shuffle[phase], 
                              num_workers[phase], pin_memory[phase])

## Initialize Regressor
Para obtener las representaciones de las descripciones en el mismo espacio de los descriptores visuales usamos dos modelos distintos. 
1. Un percptrón multicapa (MLP) de una sola capa oculta de 4096 neuronas, una capa de dropout (con probabildad 0.2) y funcion de activación Relu. 
1. Una red recurrente con una capa GRU, donde el último estado interno (de 2048 dimenciones) es usado como la representación final de las descripciones.

In [14]:
from text_encoders.regressor import MLP, RNN

regression_model_name = 'rnn' # ['mlp', 'rnn']

if regression_model_name == 'mlp':
    regression_model = MLP(in_size=text_descriptor.out_size, h_size=4096, out_size=2048)
elif regression_model_name == 'rnn':
    regression_model = RNN(in_size=text_descriptor.out_size, h_size=2048, num_layers=2, bidirectional=False, device=device)
else:
    raise 'unknown configuration: {} + {}'.format(text_descriptor_name, regression_model_name)

regression_model.to(device)
regression_model

RNN(
  (rnn): GRU(300, 2048, num_layers=2, batch_first=True, dropout=0.2)
)

## Define the Loss function
La función de pérdida mas usada para problemas de regresión es el error cuadrático medio (MSE por sus siglas en inglés). MSE es el promedio de la pérdida al cuadrado de cada ejemplo. Para calcular el MSE, sumamos todas las pérdidas al cuadrado de los ejemplos individuales y, luego, lo dividimos por la cantidad de ejemplos.

Otra funcion de perdida usada para los modelos de regresion es el error absoluto medio (MAE pos sus siglas en inglés). MAE es la suma de las diferencias absolutas entre nuestro objetivo y las variables pronosticadas. Por lo tanto, mide la magnitud promedio de los errores en un conjunto de predicciones, sin tener en cuenta sus direcciones. (Si consideramos las direcciones también, eso se llamaría Error de sesgo medio (MBE), que es una suma de residuos / errores).

En resumen, el MAE es mejor para problemas simples. Como las redes neuronales se usan generalmente para problemas complejos, esta función rara vez se usa. En adición, los descriptores visuales de nuestro problema no posee un número muy alto de dimensiones (2048) y en estos casos, usar MSE no es una limitante.

In [15]:
criterion = nn.MSELoss()
#criterion = nn.L1Loss()
#criterion = nn.SmoothL1Loss
criterion

MSELoss()

## Define the Optimizers


In [16]:
encoder_optimizer = optim.SGD(regression_model.parameters(), lr=0.01)
print(encoder_optimizer)

# if text_descriptor_name == 'my-embedding':
#     embedding_optimizer = optim.SGD(text_descriptor.parameters(), lr=0.01)
#     print(embedding_optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0
    nesterov: False
    weight_decay: 0
)


# Initialize tensorboard logger
Los resultados del entrenamiento y la evaluación los mostramos por medio de la librería TensorboardX. Esta librería permite observar en tiempo real el desempeño de los modelos mediante.

In [ ]:
!tensorboard --logdir=./log

Los resultados para cada uno de los experimentos (distintas configuraciones) se muestran en runs separados, nombrados con el formato: 

text_descriptor_name-regression_model_name-YmdHMS  

ejemplo: FastText-mlp-20190710030120

In [17]:
exp_name = '{}-{}'.format(text_descriptor_name, regression_model_name)
datetime_str = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
writer = SummaryWriter(logdir=os.path.join('./log/runs/', '{}-{}'.format(exp_name, datetime_str)))
writer

# Train Regression
A continuación entrenamos nuestro regresor y evaluamos el desempeño en cada uno de los tests de entrenamiento tras cada época. Junto con el cálculo de la pérdida en cada iteración, para la evaluación en cada época y cada test set computamos:
1. Un histograma mostrando las posiciones en que queda la imagen asociada a cada una de las 5000 descripciones.
2. La posición promedio
3. Recall at 5
4. MRR

Todos los resultados se pueden ver en el tablero de tensorboard.

In [ ]:
for e in range(20):  # epochs
    print('epoch: {}'.format(e))
    for phase in phases:
        print('phase: {}'.format(phase))
        
        regression_model.train() if phase == 'train' else regression_model.eval()
#         if text_descriptor_name == 'my-embedding':
#             text_descriptor.train() if phase == 'train' else text_descriptor.eval()
        
        names = []
        encoded_vectors = []
        loss_count = 0
        for i, (images_names, visual_feats, captions, captions_feats) in enumerate(loaders[phase]):
            with torch.set_grad_enabled(phase == 'train'):
#                 if regression_model_name == 'mlp':
#                     if text_descriptor_name == 'my-embedding':
#                         idx_texts = vocab(captions, 20)
#                         idx_texts = torch.LongTensor(idx_texts).to(device)
#                         captions_feats = text_descriptor(idx_texts)
#                         captions_feats = torch.mean(descriptors, dim=1)
#                     else:
#                         captions_feats = torch.FloatTensor(captions_feats.float()).to(device)
#                 elif regression_model_name == 'rnn':
#                     idx_texts = vocab(captions, 20)
#                     idx_texts = torch.LongTensor(idx_texts).to(device)
#                     captions_feats = text_descriptor(idx_texts)
                
                if regression_model_name == 'mlp':
                    captions_feats = torch.FloatTensor(captions_feats.float()).to(device)
                elif text_descriptor_name in ['FastText', 'GloVe']:
                    captions_feats = torch.FloatTensor(text_descriptor.transform(captions, mode='words')).to(device)
                encodes = regression_model(captions_feats)
            
                # Evaluate the loss function
                loss = criterion(encodes, visual_feats.to(device))
    
            if phase == 'train':
                loss.backward()
                encoder_optimizer.step()
#                 if text_descriptor_name == 'my-embedding':
#                     embedding_optimizer.step()
            else:
                encoded_vectors.append(encodes)
                names += images_names
            
            loss_count += loss.item()
            writer.add_scalar('{}-loss'.format(phase), loss, e * len(loaders[phase]) + i)
            if i%100 == 0:
                print('[{}/{}]'.format(i, len(loaders[phase])))
            
        print('loss: {}'.format(loss.item()/len(loaders[phase])))
    
        # compute measures
        metric,k = 'l2', 5
        if phase != 'train':                
            encoded_vectors = torch.cat(encoded_vectors, dim=0).cpu().numpy()
            visual_feats = np.array(coco_visual_feats[phase])
            
            avg_position, recall_at_k, mrr, results = 0, 0, 0, []
            for i, feats_vec in enumerate(encoded_vectors):
                if metric == 'l2':
                    dist = np.sqrt(np.sum((visual_feats - feats_vec) ** 2, axis=1))
                else:  # L1
                    dist = np.sqrt(np.sum((visual_feats - feats_vec), axis=1))
                
                sorted_idx = sorted(range(visual_feats.shape[0]), key=lambda x: dist[x])
                result_position = sorted_idx.index(coco_images_names[phase].index(names[i])) + 1
                results.append(result_position)
                avg_position += result_position
                recall_at_k += 1 if result_position <= k else 0
                mrr += 1/result_position
            writer.add_scalar('{}-avg_position'.format(phase), avg_position / len(encoded_vectors), e)
            writer.add_scalar('{}-recall@{}'.format(phase, k), recall_at_k / len(encoded_vectors), e)
            writer.add_scalar('{}-mrr'.format(phase), mrr / len(encoded_vectors), e)
            writer.add_histogram('{}-hist'.format(phase), np.array(results), e)


epoch: 0
phase: train
[0/500]
[100/500]
[200/500]
[300/500]
[400/500]
loss: 0.0015056172609329223
phase: test_A
[0/5]
loss: 0.15389121770858766
phase: test_B
[0/5]
loss: 0.14595277309417726
phase: test_C
[0/5]
loss: 0.14792768955230712
epoch: 1
phase: train
[0/500]
[100/500]
[200/500]
[300/500]
[400/500]
loss: 0.0014726345539093017
phase: test_A
[0/5]
loss: 0.15047348737716676
phase: test_B
[0/5]
loss: 0.1427962899208069
phase: test_C
[0/5]
loss: 0.14458781480789185
epoch: 2
phase: train
[0/500]


## Experiments and Results
En la carpeta ./log/runs se encuentran los logs de los siguientes experimentos:
1. FastText (centroid)  +  mlp
2. GloVe    (centroid)  +  mlp
3. tf-idf               +  mlp
4. FastText             +  rnn
5. GloVe                +  rnn

## Conclussions
A partir de los resultados obtenidos podemos concluir que el modelo que presenta las mejores propiedades de generalización es ... Esto se debe a que el uso de ..... como modelo para representar las palabras aporta ... a la regresion. Mientras que .... no es capaz de .... A su vez, los modelos basados en MLP obtuvieron mejores resultados que las RNN. Esto es una concecuencia de que las descripciones de nuestro conjunto de entrenamiento no son lo suficientemente representativas de información asociada al orden en que se presentan las palabras dentro de las oraciones y su semántica.